In [1]:

!sudo apt install -y xvfb ffmpeg
!pip install gym-contra
!pip install 'imageio=2.4.0'
!pip install PILLOW
!pip install pyglet
!pip install pyvirtualdisplay
!pip install nes_py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 783 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.7 [783 kB]
Fetched 783 kB in 1s (691 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected pack

In [2]:
from __future__ import absolute_import, division, print_function
import gym
from gym.wrappers import Monitor
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import io
import glob
import numpy as np
import PIL.Image
import pyvirtualdisplay
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 48kB/s 
     |████████████████████████████████| 3.2MB 42.5MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 491kB 45.0MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [3]:
display = Display(visible=0, size=(1400, 900))
display.start()

In [4]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [5]:
from Contra.actions import SIMPLE_MOVEMENT, COMPLEX_MOVEMENT, RIGHT_ONLY
from nes_py.wrappers import JoypadSpace


In [6]:
env_name = 'Contra-v0'
env  = gym.make(env_name)
env = JoypadSpace(env,SIMPLE_MOVEMENT)
env = wrap_env(env)
np.random.seed(123)
env.seed(123)

Make_done


[123]

In [7]:
print(env.action_space)
print(env.observation_space)

Discrete(7)
Box(0, 255, (240, 256, 3), uint8)


In [8]:
!pip install q keras==2.3.0

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

     |████████████████████████████████| 378kB 9.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [9]:
!pip install keras-rl==0.4.2

     |████████████████████████████████| 40kB 3.7MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48381 sha256=ba38ab0cfe285aa5b1503537e6c69f8b803a8e5ddf045b763a560a125befd085
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


In [10]:
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

In [11]:
nb_actions = env.action_space.n

In [12]:
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2949136   
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)               

In [13]:
memory = SequentialMemory(limit=25000, window_length=1)
policy = LinearAnnealedPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.fit(env, nb_steps=25000, visualize=True, verbose=2)

Training for 25000 steps ...

Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0


/usr/local/lib/python3.6/dist-packages/rl/memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


Score 0
Score 0


/usr/local/lib/python3.6/dist-packages/Contra/env_contra.py:153: RuntimeWarning: overflow encountered in ubyte_scalars
  _reward = self._x_position - self._x_position_last


Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0
Score 0


In [ ]:
dqn.save_weights('dqn_{}_weights.h5f'.format(env_name), overwrite=True)

In [ ]:
env.close()

In [ ]:
show_video()